In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

import os
import sys
from pathlib import Path

sys.path.insert(0, str(Path(os.getcwd()).parent.parent))

from src.common.secrets_helper import Secrets
secrets = Secrets()

# Full URL with SoQL query
url = "https://data.ny.gov/resource/wujg-7c2s.json?$query=SELECT%20transit_timestamp%2C%20transit_mode%2C%20station_complex_id%2C%20station_complex%2C%20borough%2C%20payment_method%2C%20fare_class_category%2C%20ridership%2C%20transfers%2C%20latitude%2C%20longitude%2C%20georeference%20WHERE%20%60transit_timestamp%60%20%3E%3D%20'2024-06-01'%20AND%20%60transit_timestamp%60%20%3C%20'2024-07-01'%20LIMIT%2050000"

In [2]:
# Optional: Add your App Token here if you have one
headers = {
    "Accept": "application/json",
    # TODO: edit this so the token is not displayed in raw text.
    "X-App-Token": secrets["MTA_X_APP_TOKEN"]
}

In [ ]:
offset = 0
limit = 50000
all_data = []

while True:
    paged_query = f"""
    SELECT transit_timestamp, transit_mode, station_complex_id, ...
    WHERE transit_timestamp >= '2024-06-01' AND transit_timestamp < '2024-07-01'
    LIMIT {limit} OFFSET {offset}
    """
    url = "https://data.ny.gov/resource/wujg-7c2s.json?$query=" + quote(paged_query)
    response = requests.get(url, headers=headers)
    chunk = response.json()
    
    if not chunk:
        break
    all_data.extend(chunk)
    offset += limit


In [53]:
# Send the request
response = requests.get(url, headers=headers)

# Check for successful response
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data)
    print("Success!")
else:
    print(f"Request failed with status code {response.status_code}")

# Convert the 'georeference' column to Shapely geometries
df['geometry'] = df['georeference'].apply(lambda x: shape(x) if x else None)

gdf = gpd.GeoDataFrame(df, geometry='geometry')

Success!


In [55]:
gdf.dtypes

transit_timestamp        object
transit_mode             object
station_complex_id       object
station_complex          object
borough                  object
payment_method           object
fare_class_category      object
ridership                object
transfers                object
latitude                 object
longitude                object
georeference             object
geometry               geometry
dtype: object

In [56]:
gdf.shape

(50000, 13)

In [ ]:
def api_query_string_to_sql_query(
    api_query_string: str
 )-> str:
    pass



def sql_query_to_api_query_string():
    """
    Converts a SQL query string to an API query string.
    """
    # Example SQL query
    sql_query = """
    SELECT
        transit_timestamp,
        transit_mode,
        station_complex_id,
        station_complex,
        borough,
        payment_method,
        fare_class_category,
        ridership,
        transfers,
        latitude,
        longitude,
        georeference
    WHERE transit_timestamp >= '2024-06-01' AND transit_timestamp < '2024-07-01'
    """

    # Convert SQL to API query string
    api_query_string = sql_query.replace("SELECT", "$query=SELECT").replace("WHERE", "WHERE").replace(" ", "%20").replace("'", "%27")
    
    return api_query_string


"$query=SELECT%20transit_timestamp%2C%20transit_mode%2C%20station_complex_id%2C%20station_complex%2C%20borough%2C%20payment_method%2C%20fare_class_category%2C%20ridership%2C%20transfers%2C%20latitude%2C%20longitude%2C%20georeference%20WHERE%20%60transit_timestamp%60%20%3E%3D%20'2024-06-01'%20AND%20%60transit_timestamp%60%20%3C%20'2024-07-01'"

In [21]:
from urllib.parse import quote

soql = "SELECT transit_timestamp, transit_mode, station_complex_id, station_complex, borough, payment_method, fare_class_category, ridership, transfers, latitude, longitude, georeference\nWHERE `transit_timestamp` >= '2024-06-01' AND `transit_timestamp` < '2024-07-01'"

url_2 = "https://data.ny.gov/resource/wujg-7c2s.json?$query=" + quote(soql)
url_2 == url

False

In [22]:
url_2

'https://data.ny.gov/resource/wujg-7c2s.json?$query=SELECT%20transit_timestamp%2C%20transit_mode%2C%20station_complex_id%2C%20station_complex%2C%20borough%2C%20payment_method%2C%20fare_class_category%2C%20ridership%2C%20transfers%2C%20latitude%2C%20longitude%2C%20georeference%0AWHERE%20%60transit_timestamp%60%20%3E%3D%20%272024-06-01%27%20AND%20%60transit_timestamp%60%20%3C%20%272024-07-01%27'

In [23]:
url

"https://data.ny.gov/resource/wujg-7c2s.json?$query=SELECT%20transit_timestamp%2C%20transit_mode%2C%20station_complex_id%2C%20station_complex%2C%20borough%2C%20payment_method%2C%20fare_class_category%2C%20ridership%2C%20transfers%2C%20latitude%2C%20longitude%2C%20georeference%20WHERE%20%60transit_timestamp%60%20%3E%3D%20'2024-06-01'%20AND%20%60transit_timestamp%60%20%3C%20'2024-07-01'"

In [16]:
from sodapy import Socrata
import json

MTA_APP_TOKEN = "ULqRMJBt2gpxxOgtGrRI3GXK9"

In [17]:
# ridership_fields = []

In [18]:
client = Socrata(domain="data.ny.gov", app_token=MTA_APP_TOKEN)  # Or include your app token



results = client.get(
    "wujg-7c2s",
    select="*",
    where="transit_timestamp >= '2024-06-01' AND transit_timestamp < '2024-07-01'",
    # where="transit_timestamp >= '2024-06-01' AND transit_timestamp < '2024-07-01' AND borough = 'Manhattan'",
    limit=10000000
)

with open("mta_ridership_all_boroughs_june_2024.json", "w") as file:
    json.dump(results, file)

# You can convert to DataFrame
import pandas as pd
df = pd.DataFrame.from_records(results)


In [19]:
df.shape

(2152106, 12)

In [ ]:
from shapely.geometry import shape
import geopandas as gpd

df['geometry'] = df['georeference'].apply(lambda x: shape(x) if x else None)

gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

In [25]:
gdf = gdf.set_crs("EPSG:4326")

In [26]:
gdf.to_file("mta_ridership_all_boroughs_june_2024.geojson", driver="GeoJSON")